In [1]:
!pip install openai langchain tiktoken langchain-openai langchain-community -q


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [ ]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇

In [20]:
# !wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [21]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(course_api_key=course_api_key)

# Задание 2.2.

Переделай его в шаблон так, чтобы можно было менять:

- стиль ответа
- вариант ответа, если модель не знает (вместо "Я не знаю")
- контекст
- вопрос
 И чтобы он подходил под приведённый ниже шаблон.

In [22]:
template = '''
Ответь на вопрос, исходя из контекста ниже
в стиле {style}. Если на вопрос невозможно ответить, 
используя предоставленную информацию ответь «{no_answer}».

Context: {context}

Question: {query}

Answer:
'''

In [23]:
from langchain import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["style", "no_answer", "context", "query"],
    template=template
)

In [24]:
prompt_template

PromptTemplate(input_variables=['context', 'no_answer', 'query', 'style'], input_types={}, partial_variables={}, template='\nОтветь на вопрос, исходя из контекста ниже\nв стиле {style}. Если на вопрос невозможно ответить, \nиспользуя предоставленную информацию ответь «{no_answer}».\n\nContext: {context}\n\nQuestion: {query}\n\nAnswer:\n')

# Задание 2.2.7: ⚔️ Spam 🆚 Crypto

In [25]:
import pandas as pd
from langchain import PromptTemplate
from tqdm import tqdm

In [26]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/100_crypto_messages.csv')

In [27]:
df

,text_id,text
0,143,Удаленная работа. От нас: Зп от 2500$
1,179,Что вы думаете о концепции децентрализованных ...
2,147,Удалённый заработок. Время 1-2часа в день. Мож...
3,157,Меня всегда интересовал вопрос безопасности хр...
4,168,"Я слышал, что некоторые банки начали экспериме..."
...,...,...
95,209,Работа в сфере криптовалютной журналистики или...
96,200,Какие криптовалюты вы видите в будущем как осн...
97,229,Исследование и анализ рынка криптовалют также ...
98,101,Набираю людей в команду по межбиржевой торговл...


In [34]:
# Напишите промпт, указав инструкцию, запрос и выходной индикатор
template = """Очень внимательно прочитай контекст ниже и ответь на вопрос. 
Ответом будет число 1 или 0. Если не знаешь ответ на вопрос напиши 0.

Context: {text_input}

Question: Является ли это сообщение спамом?

Answer: 
"""

prompt_template = PromptTemplate(
    input_variables=["text_input"],
    template=template
)

In [39]:
spam_labels = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['text']):
    prompt = prompt_template.format(text_input=text_input) # Добавляем сообщение в промпт
    spam_label = llm.invoke(prompt).content # Сохраняем ответ модели
    spam_labels.append(spam_label) # Добавляем ответ в список
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 100/100 [01:13<00:00,  1.37it/s]


In [38]:
df['text'].iloc[0], spam_label

('Удаленная работа. От нас: Зп от 2500$', '0')

In [40]:
df['is_spam'] = [int(e) for e in spam_labels]  # Создаём новый столбец из ответов модели

In [41]:
df.to_csv('2_2_7_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)

# Задание 2.2.8: 💼 ChatGPT - ваш ручной карьерный консультант

В данной задаче очень удобно использовать `OutputParser` (подробнее о нем в [ноутбуке](https://github.com/a-milenkin/LLM_practical_course/blob/main/notebooks/M2.2_LangChain_Prompting.ipynb))

In [42]:
import pandas as pd
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

In [43]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/vacancies_messages_50.csv')

In [45]:
df.head(5)

,text_id,text
0,9,#вакансия #vacancy #Python #удаленка #flask #r...
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...
2,28,#вакансия #vacancy #job #senior #data #DB #dat...
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...
4,18,#vacancy #job #analyst #travel #sirenatravel #...


In [46]:
df.text.iloc[0]

'#вакансия #vacancy #Python #удаленка #flask #relocation #senior\n\nПозиция: Senior Python developer\nКомпания: Collectly\nФормат работы: Удаленно (не на территории РФ/РБ)\nВилка: 6000-9000$.net\nТрудоустройство: ИП или ТК Армении\nЗанятость: fulltime\n\nКонтакты: @ann_gfio\n\nCollectly - американская технологическая компания, прошедшая акселератор Y Combinator, специализирующаяся на автоматизации биллинга в медицинской сфере США. За почти 4 года существования компания стала прибыльной и стабильной, а также приобрела статус быстрорастущей организации.\n\nCollectly - первая в мире система, обеспечивающая автономное взаимодействие американских медицинских учреждений с пациентами в отношении медицинских счетов. Компания помогает пациентам разобраться в страховых планах, оплатить счета и получить финансовую помощь, снижая операционные расходы медицинских учреждений. Работа компании Collectly решает важные социальные проблемы в США и получает признание ведущих отраслевых журналов.\n\n🔎 Кого

In [101]:
template = """Из следующего текста извлеки информацию. Если явной информации нет - пиши None:

job_title: Позиция, про которую написан текст. Нужно написать ее как в вакансии, 
без учета уровня, например если написано Senior Python developer нужно написать Python developer.

company: Название компании, которая нанимает человека. Только название, как в тексте.

salary: Зарплата.
Может быть написано в виде тысяч: 100к или 100 тыс. - надо умножить на 1000 и оставить только число.
Например на 100к или 100 тыс. надо написать 100000.
Не пиши net/gross/на руки и.т.д.
Если в тексте указан диапазон пиши два числа через тире без пробелов между тире и числами.
Если указана только нижняя граница, то пиши 'от' и число, если только верхняя - то 'до' и число.
После числа или диапазона надо написать 'руб.' или '$' через пробел в зависимости от валюты.
Если зарплата указана в час, то надо после всего через пробел добавить 'в час'.

tg: Контакт телеграм, с которым нужно связаться, чтобы откликнуться. 
Пишется вот так: @<контакт> . Если указано больше 1 контакта надо вывести все в списке.

grade: Уровень знаний кандидата. 
Вот возможные варианты в порядке возрастания: intern, junior, junior+, middle, middle+, senior, lead
Может быть написано на русском языке, например ведущий это то же самое что и lead
Если указано несколько, напиши через запятую в порядке возрастания.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)

In [102]:
job_title_schema = ResponseSchema(
    name="job_title",
    description="""Позиция, про которую написан текст. Нужно написать ее как в вакансии, 
    без учета уровня, например если написано Senior Python developer нужно написать Python developer
    """
)


company_schema = ResponseSchema(
    name="company",
    description="""Название компании, которая нанимает человека. Только название, как в тексте.
    """
)


salary_schema = ResponseSchema(
    name="salary",
    description="""Зарплата.
    Может быть написано в виде тысяч: 100к или 100 тыс. - надо умножить на 1000 и оставить только число.
    Например на 100к или 100 тыс. надо написать 100000.
    Не пиши net/gross/на руки и.т.д.
    Если в тексте указан диапазон пиши два числа через тире без пробелов между тире и числами.
    Если указана только нижняя граница, то пиши 'от' и число, если только верхняя - то 'до' и число.
    После числа или диапазона надо написать 'руб.' или '$' через пробел в зависимости от валюты.
    Если зарплата указана в час, то надо после всего через пробел добавить 'в час'."""
)


tg_schema = ResponseSchema(
    name="tg",
    description="""Контакт телеграм, с которым нужно связаться, чтобы откликнуться. 
    Пишется вот так: @<контакт> . Если указано больше 1 контакта надо вывести все в списке. """
)


grade_schema = ResponseSchema(
    name="grade",
    description="""Уровень знаний кандидата. 
    Вот возможные варианты в порядке возрастания: intern, junior, junior+, middle, middle+, senior, lead
    Может быть написано на русском языке, например ведущий это то же самое что и lead
    Если указано несколько, напиши через запятую в порядке возрастания."""
)

response_schemas = [
    job_title_schema,
    company_schema,
    salary_schema,
    tg_schema,
    grade_schema
]

In [103]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas) # Создаём парсер и подаём в него список со схемами
format_instructions = output_parser.get_format_instructions() # Получаем инструкции по форматированию ответа

In [107]:
dict_list = [] # Список из словарей, где будем хранить полученную информацию
for text_input in tqdm(df['text']):
    messages = prompt.format_messages(
        text=text_input,
        format_instructions=format_instructions
    )
    response = llm.invoke(messages) # Ответ модели
    dict_list.append(output_parser.parse(response.content)) # Переводим содержимое ответа модели в словарь и добавляем в список
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 50/50 [01:13<00:00,  1.47s/it]


In [108]:
result_df = pd.DataFrame(dict_list) # Создаем датафрейм из полученной информации
ans_df = pd.concat([df, result_df], axis=1) # Объединяем всё в один датафрейм
ans_df.to_csv('2_2_8_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)

In [110]:
ans_df

,text_id,text,job_title,company,salary,tg,grade
0,9,#вакансия #vacancy #Python #удаленка #flask #r...,Python developer,Collectly,6000-9000 $,@ann_gfio,middle
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...,Системный аналитик,АО «ГНИВЦ»,320000-370000 руб.,@NatalyaMaki,None
2,28,#вакансия #vacancy #job #senior #data #DB #dat...,Database Administrator,Match Systems,от 3000 $,@lex_kertis,senior
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...,Руководитель отдела системного администрирования,финтех компания,от 3000 $,@ResearcherRIT,"middle, senior"
4,18,#vacancy #job #analyst #travel #sirenatravel #...,Analyst,Sirena Travel,120000-180000 руб.,@ann_gfio,middle
5,48,#вакансия #fulltime #remote #lookfor #devops\n...,DevOps инженер,Mad Devs,до 5000 $,@recruiter_maddevs,senior
6,41,#вакансия #senior #middle+ #lead #удаленка #оф...,Системный аналитик,Платформа,180000-300000 руб.,@Alexandrabogdanova_96,"middle+, senior, lead"
7,6,#job #python #django #javascript #react #fulls...,Python/Django full-stack разработчика,ivelum,4000-6000 $,@lebedevaoi,middle
8,17,#вакансия #vacancy #middle #senior #remote #уд...,Go разработчик,КА Алешин Д.А.,до 500000 руб.,@AleshinDmitry80,"middle, senior"
9,4,Вакансия: Разработчик (Vue.js / Golang) для на...,Разработчик,None,None,None,junior
